# Calculating molecular orbitals

In this notebook, developed by Anders Lervik from the Norwegian University of Science and Technology (https://www.andersle.no) and with the original available from his GitHub repository: (https://github.com/andersle/andersleno/blob/main/2022/mo/ethene.ipynb), we will perform Hartree Fock calculations with pyscf. We will use geometries that RDKit generates, and will not optimize the geometry with pyscf here to save some computational time.

We will calculate and visualize the so-called canonical Hartree-Fock molecular orbitals, which are composed of linear combination of atomic orbitals.

To create molecules, we will draw then first in 2D, using an automatic SMILES generator that runs in a webbrowser: https://www.cheminfo.org/flavor/malaria/Utilities/SMILES_generator___checker/index.html

We will export the structures into a so-called SMILES code, a string-based descriptor for 2D representaton of the molecular structure.

In this tutorial, we will use the RDKit tools to convert the SMILES code into a 3D structure, for which we will compute the molecular orbitals. We will start very easy with a water molecule, for which the SMILES code is just "O".

However, before we can befin, we will need to install a ton of libraries. Will will do that automaticallu with pip... No wories, you can just shift-enter your way through the next entries!

In [ ]:
!pip3 install rdkit py3Dmol pythreejs fortecubeview pyscf seaborn matplotlib scikit-image

In [ ]:
!pip3 install seaborn matplotlib scikit-image

Next, we import [RDKit](https://www.rdkit.org/) (used for generating coordinates),
[py3Dmol](https://pypi.org/project/py3Dmol/)
and [fortecubeview](https://github.com/evangelistalab/fortecubeview) (used for visualization),
[matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) (used for plotting),
[pandas](https://pandas.pydata.org/) and [numpy](https://numpy.org/) (used for some numerics results),
and [pyscf](https://pyscf.org/) (used for the quantum mechanics).

In [ ]:
import pathlib

# RDKit imports:
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    rdCoordGen,
)
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # Use higher quality images for molecules

# For visualization of molecules and orbitals:
import py3Dmol
import fortecubeview

# pyscf imports:
from pyscf import gto, scf, lo, tools

# For plotting
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme(style="ticks", context="talk", palette="muted")

# For numerics:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:,.3f}".format

## Set up the molecule with RDKit

For this example, I will use water - it is a simple molecule and from basic chemistry; bit boring perhaps... 
The SMILES string for water (H$_2$O) is just a single letter "O".

In [ ]:
molecule_name = "water"
molecule = Chem.MolFromSmiles("O")  # Generate the molecule from smiles
molecule

In [ ]:
def get_xyz(molecule, optimize=False):
    """Get xyz-coordinates for the molecule"""
    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)
    if optimize:  # Optimize the molecules with the MM force field:
        AllChem.MMFFOptimizeMolecule(mol)
    xyz = []
    for lines in Chem.MolToXYZBlock(mol).split("\n")[2:]:
        strip = lines.strip()
        if strip:
            xyz.append(strip)
    xyz = "\n".join(xyz)
    return mol, xyz

In [ ]:
molecule3d, xyz = get_xyz(molecule)

In [ ]:
view = py3Dmol.view(
    data=Chem.MolToMolBlock(molecule3d),
    style={"stick": {}, "sphere": {"scale": 0.3}},
    width=300,
    height=300,
)
view.zoomTo()

## Run pyscf and calculate molecular orbitals

In [ ]:
def run_calculation(xyz, basis="sto-3g"):
    """Calculate the energy (+ additional things like MO coefficients) with pyscf."""
    mol = gto.M(
        atom=xyz,
        basis=basis,
        unit="ANG",
        symmetry=True,
        charge = 0,
        spin = 0,
    )
    mol.build()
    mf = scf.RHF(mol).run()
    return mf, mol

In [ ]:
mf, mol = run_calculation(xyz, basis="sto-3g")

To show some results from the calculation (for instance energies and contributions to the molecular orbitals)
we can make use of `mf.analyze`.
This is commented out here to reduce the output.

In [ ]:
#mf.analyze(verbose=5)

We can access the energy and occupancy of the molecular orbitals via the `.mo_energy`
and `.mo_occ` attributes:

In [ ]:
table = pd.DataFrame({"Energy": mf.mo_energy, "Occupancy": mf.mo_occ})
table

Let us also make a plot of the energy levels:

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(9, 6))
#colors = matplotlib.cm.get_cmap("tab20")(np.linspace(0, 1, len(mf.mo_energy)))
colors = matplotlib.colormaps.get_cmap("tab20")(np.linspace(0, 1, len(mf.mo_energy)))

pos = []
for i, (energy, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
    left = 3 * i
    right = 3 * i + 2.5
    length = right - left

    (line,) = ax.plot([left, right], [energy, energy], color=colors[i], lw=3)

    electron_x, electron_y = None, None
    if occ == 2:
        electron_x = [left + 0.25 * length, left + 0.75 * length]
        electron_y = [energy, energy]
    elif occ == 1:
        electron_x, electron_y = [left + 0.5], [energy]
    if electron_x and electron_y:
        ax.scatter(electron_x, electron_y, color=line.get_color())

    pos.append(left + 0.5 * length)

ax.axhline(y=0, ls=":", color="k")
ax.set_xticks(pos)
ax.set_xticklabels([f"#{i}" for i, _ in enumerate(pos)])
ax.set(xlabel="MO number", ylabel="Energy / a.u.")
sns.despine(fig=fig)

Now, we have the energies and occupancy. Next, we will display the orbitals and look at them. The canonical ones are
available via `mf.mo_coeff` with the routine below:

In [ ]:
def get_mo(mf, mol):
    """Get molecular orbitals"""
    orbitals = {"canonical": mf.mo_coeff}
    return orbitals

In [ ]:
orbitals = get_mo(mf, mol)

## Visualizing the orbitals with fortecubeview

For visualizing the molecular orbitals, it is convenient to write them as [cube files](http://paulbourke.net/dataformats/cube/). These cube files can be
visualized in jupyter by [py3Dmol](https://pypi.org/project/py3Dmol) or [fortecubeview](https://github.com/evangelistalab/fortecubeview), or in external programs such as [VMD](https://www.ks.uiuc.edu/Research/vmd/).

Here is a short method to write the cube files for some given molecular orbital coefficients:

In [ ]:
def write_all_coeffs(
    mol, coeffs, prefix="cmo", dirname=".", margin=5, offset=0
):
    """Write cube files for the given coefficients."""
    path = pathlib.Path(dirname)
    path.mkdir(parents=True, exist_ok=True)

    for i in range(coeffs.shape[1]):
        outfile = f"{prefix}_{i+offset:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, coeffs[:, i], margin=margin)

To write all the canonical molecular orbitals `write_all_coeffs` into a subdirectory "dirname", can be used as follows:

In [ ]:
write_all_coeffs(
    mol,
    orbitals["canonical"],
    prefix=f"{molecule_name}_cmo",
    dirname="water",
    margin=5,
)

To display the molecular orbitals, [fortecubeview](https://github.com/evangelistalab/fortecubeview) is really convenient as we can give it a directory, and it will load all the cube files in that directory:

In [ ]:
fortecubeview.plot(path="./idontknow", sumlevel=0.85)